In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import time
from datetime import datetime
from urllib.error import HTTPError
import requests

In [ ]:
df = pd.read_csv("links.csv", index_col=[0])
df = df.drop_duplicates( keep='last')
df = df.reset_index(drop=True)

In [ ]:
df['location'] = df['link'].str.split('/').str[1]
df['type'] = df['link'].str.split('/').str[3]

In [ ]:
hdr = {'User-Agent': 'Mozilla/5.0'}
price_df = []
size_df = []
rooms_df = []
num = 1
for i in df['link']:
    site = "https://ingatlan.com" + str(i)
    
    try:
        response = requests.get(site, headers=hdr)
        response.raise_for_status()
    except HTTPError as hp:
        print(hp)
     
    else:
        site= "https://ingatlan.com" + str(i)
        req = Request(site,headers=hdr)
        page = urlopen(req)
        soup = BeautifulSoup(page)
        price = soup.find_all('span', {'fw-bold fs-5 text-nowrap'})
        price = price[0].get_text()
        price = price.strip()
        price_df.append(price)

        rooms = soup.find_all('span', {'fw-bold fs-5'})
        rooms = rooms[1].get_text()
        rooms = rooms.strip()
        rooms_df.append(rooms)

        size = soup.find_all('span', {'fw-bold fs-5'})
        size = size[0].get_text()
        size = size.strip()
        size_df.append(size)
        
        print('done: ' + str(num) + " from " + str(len(df)))
        num = num + 1
        #time.sleep(2.5) #dont want to overwhelm the site

In [ ]:
df['size'] = size_df
df['rooms'] = rooms_df
df['price'] = price_df

In [ ]:

now = datetime.now() # current date and time
date = now.strftime("%Y-%m-%d")
df.to_csv("dataframe-" + date + ".csv", sep = ";")